### MID TERM

In [1]:
import os
import pandas as pd
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

In [3]:
all_files = [os.path.join('../data/waitrose',file) for file in os.listdir('../data/waitrose')
             if file.endswith('.csv')]
all_files

['../data/waitrose\\baby-child-and-parent.csv',
 '../data/waitrose\\bakery.csv',
 '../data/waitrose\\beer-wine-and-spirits.csv',
 '../data/waitrose\\best-of-british.csv',
 '../data/waitrose\\dietary-and-lifestyle.csv',
 '../data/waitrose\\everyday-value.csv',
 '../data/waitrose\\food-cupboard.csv',
 '../data/waitrose\\fresh-and-chilled.csv',
 '../data/waitrose\\frozen.csv',
 '../data/waitrose\\home.csv',
 '../data/waitrose\\household.csv',
 '../data/waitrose\\new.csv',
 '../data/waitrose\\organic-shop.csv',
 '../data/waitrose\\pet.csv',
 '../data/waitrose\\summer.csv',
 '../data/waitrose\\tea-coffee-and-soft-drinks.csv',
 '../data/waitrose\\toiletries-health-and-beauty.csv',
 '../data/waitrose\\waitrose-brands.csv']

In [4]:
df = pd.concat((pd.read_csv(file)for file in all_files))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25418 entries, 0 to 1593
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data-product-id        25418 non-null  int64  
 1   data-product-name      25418 non-null  object 
 2   data-product-type      25418 non-null  object 
 3   data-product-on-offer  25418 non-null  bool   
 4   data-product-index     25408 non-null  float64
 5   image-url              25418 non-null  object 
 6   product-page           25418 non-null  object 
 7   product-name           25407 non-null  object 
 8   product-size           25363 non-null  object 
 9   item-price             25407 non-null  object 
 10  price-per-unit         24976 non-null  object 
 11  offer-description      7201 non-null   object 
 12  category               25418 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 2.5+ MB


In [7]:
df = df.drop_duplicates()

df = df.drop(columns=['data-product-name', 
                      'data-product-type', 
                      'data-product-index'])


In [8]:
df = (
    df.rename(columns={
        'data-product-id': 'id',
        'data-product-on-offer': 'offer',
        'product-page': 'page',
        'product-name': 'name',
        'product-size': 'size',
    })
)

In [9]:
df['id'] = df['id'].astype('int32')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25378 entries, 0 to 1593
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 25378 non-null  int32 
 1   offer              25378 non-null  bool  
 2   image-url          25378 non-null  object
 3   page               25378 non-null  object
 4   name               25367 non-null  object
 5   size               25323 non-null  object
 6   item-price         25367 non-null  object
 7   price-per-unit     24936 non-null  object
 8   offer-description  7178 non-null   object
 9   category           25378 non-null  object
dtypes: bool(1), int32(1), object(8)
memory usage: 1.9+ MB


#### TASK 01-DATA CLEANING

In [12]:
df['item-price']

0                  £3.15
1                  £3.15
2                 £13.50
3                    95p
4                  £1.60
              ...       
1589    £24.31 each est.
1590               £2.40
1591              £10.00
1592               £2.50
1593              £21.49
Name: item-price, Length: 25378, dtype: object

In [13]:
def clean_item_price(item_price: str):
   if 'each est.' in item_price:
      item_price = item_price.replace('each est.','')
   if type(item_price) != str:  
     pass
   elif '£'in item_price:
      item_price = item_price.replace('£','')
   elif 'p' in item_price:
     item_price = item_price.replace('p','')
     item_price = '0.' + item_price
   if'-'in item_price:
      item_price = item_price.split('-')[0]
   return float(item_price)

In [14]:
df['item-price-fixed'] = df['item-price'].astype(str).apply(clean_item_price)

In [15]:
df['item-price-fixed']

0        3.15
1        3.15
2       13.50
3        0.95
4        1.60
        ...  
1589    24.31
1590     2.40
1591    10.00
1592     2.50
1593    21.49
Name: item-price-fixed, Length: 25378, dtype: float64

In [16]:
selected_row = df['item-price'].astype(str).str.contains('-')

In [17]:
df.groupby('name')['category'].apply(list)

name
019521 - Essential Flageolet Beans in Water      [Dietary & Lifestyle, Food Cupboard, Waitrose ...
035738 - Solero Exotic Ice cream Lolly                       [Dietary & Lifestyle, Frozen, Summer]
044889 - Essential Double Cream Large                           [Fresh & Chilled, Waitrose Brands]
060167 - Kronenbourg 1664 Cans                                              [Beer, Wine & Spirits]
086143 - Cooks' Ingredients Garlic               [Dietary & Lifestyle, Fresh & Chilled, Waitros...
                                                                       ...                        
£100,000 a Month for a Year Scratchcard                                                     [Home]
Öpso Japanese Sakura Tree Reed Diffuser                                          [Home, Household]
Öpso Mediterranean Citrus Grove Reed Diffuser                                    [Home, Household]
Öpso Nordic Birch Forest Reed Diffuser                                           [Home, Household]
Ürzig

In [18]:
categories = df.groupby('name')['category'].apply(list)

In [19]:
df['item-price'] = df['item-price-fixed']
df.drop(columns=['item-price-fixed'],inplace=True)

In [20]:
df[df['category'].str.contains('everyday',case=False)]

,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
0,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,Everyday Value
1,86412,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Red Peppers,Typical weight 0.16kg,0.60,£3.75/kg,NaN,Everyday Value
2,88528,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Fairtrade Bananas,5s,0.95,19p each,NaN,Everyday Value
3,85115,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Onions,Typical weight 0.15kg,0.15,£1/kg,NaN,Everyday Value
4,88411,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Lemons,Each,0.35,35p each,NaN,Everyday Value
...,...,...,...,...,...,...,...,...,...,...
136,551848,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential 24 British Beef Meatballs,400g,3.60,£9/kg,NaN,Everyday Value
137,776778,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Kitchen Wipes,36s,1.10,3.1p each,NaN,Everyday Value
138,6745,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Mixed Fruit,1kg,3.25,£3.25/kg,NaN,Everyday Value
139,873709,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Original Instant Oats,10x27g,1.35,50p/100g,NaN,Everyday Value


In [21]:
df[~df['category'].str.contains('everyday',case=False)]

,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
0,525635,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
1,557746,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent"
3,767801,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent"
4,514054,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent"
...,...,...,...,...,...,...,...,...,...,...
1589,18427,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic British Lamb Half Leg,Typical weight 0.94kg,24.31,£26.00/kg,NaN,Waitrose Brands
1590,11606,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients Wok Oil,250ml,2.40,96p/100ml,NaN,Waitrose Brands
1591,6903,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no-1-ba...,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,350g,10.00,£28.58/kg,NaN,Waitrose Brands
1592,6125,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients White Marzipan,500g,2.50,£5/kg,NaN,Waitrose Brands


In [22]:
dd=df[df['category'].str.contains('everyday',case=False,)].head(5)

In [23]:
dd.sort_values(by = 'item-price', ascending=True).head(5)

,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
3,85115,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Onions,Typical weight 0.15kg,0.15,£1/kg,NaN,Everyday Value
4,88411,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Lemons,Each,0.35,35p each,NaN,Everyday Value
1,86412,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Red Peppers,Typical weight 0.16kg,0.60,£3.75/kg,NaN,Everyday Value
0,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,Everyday Value
2,88528,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Fairtrade Bananas,5s,0.95,19p each,NaN,Everyday Value


In [24]:
dd

,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
0,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,Everyday Value
1,86412,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Red Peppers,Typical weight 0.16kg,0.60,£3.75/kg,NaN,Everyday Value
2,88528,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Fairtrade Bananas,5s,0.95,19p each,NaN,Everyday Value
3,85115,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Onions,Typical weight 0.15kg,0.15,£1/kg,NaN,Everyday Value
4,88411,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Lemons,Each,0.35,35p each,NaN,Everyday Value


In [25]:
everyday = dd[['id','name','size','item-price','category']]

In [26]:
everyday

,id,name,size,item-price,category
0,86468,Essential Cucumber,Each,0.95,Everyday Value
1,86412,Essential Red Peppers,Typical weight 0.16kg,0.60,Everyday Value
2,88528,Essential Fairtrade Bananas,5s,0.95,Everyday Value
3,85115,Essential Onions,Typical weight 0.15kg,0.15,Everyday Value
4,88411,Essential Lemons,Each,0.35,Everyday Value


In [27]:
dm=df[~df['category'].str.contains('everyday',case=False)].head(5)
dm.sort_values(by = 'item-price', ascending=True)

,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
3,767801,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent"
4,514054,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent"
1,557746,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
0,525635,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent"


In [28]:
brand= dm[['id','name','size','item-price','category']]

In [29]:
brand

,id,name,size,item-price,category
0,525635,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,"Baby, Child & Parent"
1,557746,Organix Carrot Cake Oaty Bars,6x23g,3.15,"Baby, Child & Parent"
2,32062,Aptamil 2 Follow On Milk,800g,13.50,"Baby, Child & Parent"
3,767801,Essential Baby Wipes,80s,0.95,"Baby, Child & Parent"
4,514054,Organix Apple Rice Cakes,40g,1.60,"Baby, Child & Parent"


In [37]:
dm

,id,offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
0,525635,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
1,557746,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent"
3,767801,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent"
4,514054,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent"


In [38]:
_categories = df.sort_values(by = 'item-price', ascending=True).head(5)

In [39]:
branded = pd.concat([everyday, brand], ignore_index=True)

In [40]:
branded

,id,name,size,item-price,category
0,86468,Essential Cucumber,Each,0.95,Everyday Value
1,86412,Essential Red Peppers,Typical weight 0.16kg,0.60,Everyday Value
2,88528,Essential Fairtrade Bananas,5s,0.95,Everyday Value
3,85115,Essential Onions,Typical weight 0.15kg,0.15,Everyday Value
4,88411,Essential Lemons,Each,0.35,Everyday Value
5,525635,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,"Baby, Child & Parent"
6,557746,Organix Carrot Cake Oaty Bars,6x23g,3.15,"Baby, Child & Parent"
7,32062,Aptamil 2 Follow On Milk,800g,13.50,"Baby, Child & Parent"
8,767801,Essential Baby Wipes,80s,0.95,"Baby, Child & Parent"
9,514054,Organix Apple Rice Cakes,40g,1.60,"Baby, Child & Parent"


In [41]:
ggplot (branded, aes(x='name',y='item-price',fill='category')) + \
    geom_bar(star='identify') + \
    ggtitle('Essential Product Price') + \
    xlab('Name') + \
    ylab('Item Price') + \
    theme (axis_text_x=element_text(angle=45, hjust=1),
           )+\
    ggsize(800,500)

In [43]:
ggplot(branded, aes(x='name', y='item-price',color='category')) + \
    geom_line( size=2) + \
    geom_point(size=5)+\
    ggtitle('All Brand Price') + \
    xlab('Name') + \
    ylab('Item Price')+\
    ggsize(1000,800)
    
    
